In [1]:
import tensorflow as tf
import numpy as np
from utils.utils import voxel_grid
import os
from sklearn.preprocessing import LabelBinarizer, OneHotEncoder



In [65]:
#Parameters

RES = (64,64,64)
BATCH_SIZE = 12


#Helper functions (layer creation)

"""
    Default stride size is filterSize, which means no overlapping.
"""
def conv3d(x, numFilters, filterSize, stride=False, padding='VALID'):
    if(not stride):
        stride = filterSize
    inChannels = x.get_shape()[-1].value
    with tf.name_scope("conv"):
        #The filter weights must be of the form (width, height, depth, channels, outputChannels[numberOfFilters])
        filter=tf.Variable(tf.truncated_normal(shape=[3,3,3,inChannels,numFilters], dtype=tf.float32))
        return tf.nn.conv3d(input=x, filter=filter, \
                            strides=[1,stride,stride,stride,1], padding=padding)


def flatten(x):
    return tf.reshape(x, [batch_size, -1])

def fully_connected(x, size, activation=tf.nn.relu):
    with tf.name_scope("fully_connected"):
        b = tf.Variable(tf.zeros([size]), validate_shape=False)
        weights = tf.Variable(tf.truncated_normal([batch_size, size]), validate_shape=False)
        return activation(b + tf.matmul(x, weights))


#Helper functions batch reading

object_csvs = list(filter(lambda x: ".csv" in x, os.listdir("../dataset/objects")))
labels = np.array(list(map(lambda x: x.split(".")[0], object_csvs)))
labels = labels.reshape((-1,1))

le = LabelBinarizer(sparse_output=False)

labels = le.fit_transform(labels).reshape(-1,26)

def get_batch(size=BATCH_SIZE, ind=0):
    
    mini_batch_x = []
    mini_batch_y = []
    for i in range(size):
        obj = voxel_grid(np.genfromtxt("../dataset/objects/"+object_csvs[ind+i], delimiter=",")[:,3:6], res=RES)
        mini_batch_x.append(obj[:,:,:,None])
        #mini_batch_x[i].append(object_csvs[ind+i].split(".")[0])
        #write code for labels
    
    
    return (np.array(mini_batch_x), labels[ind:ind+size])


In [66]:
#Build model


#Architecture

x = tf.placeholder(shape=[None,RES[0], RES[1], RES[2],1], dtype=tf.float32)
y = tf.placeholder(shape=[None, labels[0].size], dtype=tf.float32)


batch_size = tf.shape(x)[0] 

print(x.get_shape())
print(y.get_shape())

conv1 = conv3d(x, 3,3,2)
conv2 = conv3d(conv1, 3,3,2)

output = fully_connected(flatten(conv2), labels[0].size, activation=tf.nn.softmax)

print(output.get_shape())

#Loss
loss = tf.nn.softmax_cross_entropy_with_logits(output, y)
optimizer = tf.train.AdamOptimizer(0.5)


train_step = optimizer.minimize(loss)









(?, 64, 64, 64, 1)
(?, 26)
<unknown>


ValueError: as_list() is not defined on an unknown TensorShape.

# Model Training

Here comes the model training

In [42]:
with tf.Session() as sess:
    sess.run(tf.initialize_all_variables())
    
    batch_x, batch_y = get_batch(12,0)
    print(batch_y.shape)
    feed_dict = {
        x: batch_x,
        y: batch_y
    }
    for i in range(100):
        
        loss, _ = sess.run([loss, train_step], feed_dict=feed_dict)
        if i%10 == 0:
            print("Loss is: %.2f" % loss)
    
        
    
    

Instructions for updating:
Use `tf.global_variables_initializer` instead.
(12, 26)


InvalidArgumentError: Input to reshape is a tensor with 121500 values, but the requested shape requires a multiple of 262144
	 [[Node: Reshape_29 = Reshape[T=DT_FLOAT, Tshape=DT_INT32, _device="/job:localhost/replica:0/task:0/cpu:0"](conv_24/Conv3D, Reshape_29/shape)]]

Caused by op 'Reshape_29', defined at:
  File "/Users/Jimmy/anaconda3/envs/tensorflow/lib/python3.5/runpy.py", line 184, in _run_module_as_main
    "__main__", mod_spec)
  File "/Users/Jimmy/anaconda3/envs/tensorflow/lib/python3.5/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/Users/Jimmy/anaconda3/envs/tensorflow/lib/python3.5/site-packages/ipykernel/__main__.py", line 3, in <module>
    app.launch_new_instance()
  File "/Users/Jimmy/anaconda3/envs/tensorflow/lib/python3.5/site-packages/traitlets/config/application.py", line 658, in launch_instance
    app.start()
  File "/Users/Jimmy/anaconda3/envs/tensorflow/lib/python3.5/site-packages/ipykernel/kernelapp.py", line 474, in start
    ioloop.IOLo